In [ ]:
# imports
from csv import DictReader, DictWriter
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
og_column_names = ['station_id', 'longitude', 'latitude', 'time', 'AtmospherePressure', 'WindDirection', 'WindSpeed', 'Gust', 'WaveHeight', 'WavePeriod', 'MeanWaveDirection', 'Hmax', 'AirTemperature', 'DewPoint', 'SeaTemperature', 'RelativeHumidity']

column_names = ['AtmospherePressure', 'WindDirection', 'WindSpeed', 'Gust', 'AirTemperature', 'SeaTemperature', 'RelativeHumidity', 'WaveHeight', 'WavePeriod']

with open('../data/raw_data.csv', 'r') as f1, open('../data/clean_data.csv', 'w') as f2:
    reader = DictReader(f1, fieldnames=og_column_names)
    writer = DictWriter(f2, fieldnames=column_names, lineterminator='\n')

    next(reader)
    next(reader)

    writer.writeheader()

    for line in tqdm(reader):
        if any(line[column_name] == 'NaN' for column_name in column_names):
            continue
        writer.writerow({column_name: line[column_name] for column_name in column_names})


In [ ]:
df = pd.read_csv('../data/clean_data.csv')
df.describe()

In [ ]:
# Use the interquartile range to remove the outliers in data.
for column_name in column_names:

    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1

    lower_whisker = Q1 - 1.5*IQR
    upper_whisker = Q3 + 1.5*IQR
    df = df[(df[column_name] >= lower_whisker) & (df[column_name] <= upper_whisker)]

df.describe()

In [ ]:
features = column_names[:-2]
labels = column_names[-2:]

X = df[features].to_numpy()
# Note that we have 2 labels
Y = df[labels].to_numpy()

In [ ]:
# The commonly used split of 75% for training, 10% for validation, and 15%
# for the test sets has been followed when splitting the dataset.
# V Roshan Joseph. Optimal ratio for data splitting. Statistical Analysis and Data Mining: The
# ASA Data Science Journal, 15(4):531–538, 2022.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.6, random_state=42)

# y1: WaveHeight
y1_train = Y_train[:,0]
y1_val = Y_val[:,0]
y1_test = Y_test[:,0]

# y2: WavePeriod
y2_train = Y_train[:,1]
y2_val = Y_val[:,1]
y2_test = Y_test[:,1]

In [ ]:
def finding_polynomial(X_train, X_val, y_train, y_val, highest_degree=3):
    # define a list of values for polynomial degrees
    degrees = [i for i in range(1, highest_degree + 1)]
    # declare a variable to store the resulting validation errors for each polynomial degree
    val_errors = []
    for i in range(len(degrees)):    # use for-loop to fit polynomial regression models with different degrees

        poly = PolynomialFeatures(degree=degrees[i])
        X_train_poly = poly.fit_transform(X_train)
        X_val_poly = poly.fit_transform(X_val)

        regression = LinearRegression()
        regression.fit(X_train_poly, y_train)

        y_pred = regression.predict(X_val_poly)

        val_error = mean_squared_error(y_val, y_pred)

        val_errors.append(val_error)

    return val_errors

In [ ]:
val_errors_1 = finding_polynomial(X_train, X_val, y1_train, y1_val)
val_errors_2 = finding_polynomial(X_train, X_val, y2_train, y2_val)
print(val_errors_1)
print(val_errors_2)